# Amazon SageMaker Model Governance - Model Cards Model Registry integration

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

---

This notebook walks you through the new feature of Amazon SageMaker Model Cards Model Registry Integration. To learn about the existing features and for more information on model cards, see [Model Cards](https://docs.aws.amazon.com/sagemaker/latest/dg/model-cards.html) in the _Amazon SageMaker Developer Guide_.

Amazon SageMaker Model Cards give you the ability to create a centralized, customizable fact-sheet to document critical details about your machine learning (ML) models. Use model cards to keep a record of model information, such as intended uses, risk ratings, training details, evaluation metrics, and more for streamlined governance and reporting. 

In this example, you will create a model package along with a model card to document model package details along the way. Learn how to create a model card by associating model package using the Amazon SageMaker Python SDK.

---
## Contents

1. [Setup](#Setup)
1. [Create a model package](#ModelPackage)
1. [Create Model Card](#ModelCard)
1. [Cleanup](#Cleanup)

---
## Setup
To begin, you must specify the following information:
- The IAM role ARN used to give SageMaker training and hosting access to your data. The following example uses the SageMaker execution role.
- The SageMaker session used to manage interactions with Amazon SageMaker Model Card API methods.
- The S3 URI (`bucket` and `prefix`) where you want to store training artifacts, models, and any exported model card PDFs. This S3 bucket should be in the same Region as your Notebook Instance, training, and hosting configurations. The following example uses the default SageMaker S3 bucket and creates a default SageMaker S3 bucket if one does not already exist.
- The S3 session used to manage interactions with Amazon S3 storage.

In [ ]:
! pip install --upgrade sagemaker

In [ ]:
import boto3
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker_session = Session()

bucket = sagemaker_session.default_bucket()
prefix = "model-card-registry-sample-notebook"
region = sagemaker_session.boto_region_name

print(bucket)
print(region)

Next, import the necessary Python libraries.

In [ ]:
import io
import os
import time
import numpy as np
from six.moves.urllib.parse import urlparse
from pprint import pprint
import sagemaker
from sagemaker.image_uris import retrieve
import sagemaker.amazon.common as smac
from sagemaker.model_card import (
    ModelCard,
    ModelPackage,
    IntendedUses,
    ModelCardStatusEnum,
)
from sagemaker.model_card.model_card import ModelApprovalStatusEnum

---
## Prepare a Model (Model package)<a name="ModelPackage"></a>
The following code creates an example model package trained on a synthetic dataset. The target variable (0 or 1) is the second variable in the tuple.

### 1. Prepare the training data
The code will upload example data to your S3 bucket.

In [ ]:
s3 = boto3.client("s3", region_name=region)

# synthetic data
raw_data = (
    (0.5, 0),
    (0.75, 0),
    (1.0, 0),
    (1.25, 0),
    (1.50, 0),
    (1.75, 0),
    (2.0, 0),
    (2.25, 1),
    (2.5, 0),
    (2.75, 1),
    (3.0, 0),
    (3.25, 1),
    (3.5, 0),
    (4.0, 1),
    (4.25, 1),
    (4.5, 1),
    (4.75, 1),
    (5.0, 1),
    (5.5, 1),
)
training_data = np.array(raw_data).astype("float32")
labels = training_data[:, 1]

# upload data to S3 bucket
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, training_data, labels)
buf.seek(0)
boto3.resource("s3").Bucket(bucket).Object(os.path.join(prefix, "train")).upload_fileobj(buf)

### 2. Create a training job
Train a binary classification model with the training data from the previous step.

In [ ]:
s3_train_data = f"s3://{bucket}/{prefix}/train"
output_location = f"s3://{bucket}/{prefix}/output"
container = retrieve("linear-learner", sagemaker_session.boto_session.region_name)
estimator = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
)
estimator.set_hyperparameters(feature_dim=2, mini_batch_size=10, predictor_type="binary_classifier")
estimator.fit({"train": s3_train_data})
print(f"Training job name: {estimator.latest_training_job.name}")

### 2. Create a model package

In [ ]:
# create a model package group
model_package_group_name = "test-notebook-model-package-group"
sagemaker_session.sagemaker_client.create_model_package_group(
    ModelPackageGroupName=model_package_group_name
)

# describe training job to get model_data_url and image
training_job_name = estimator.latest_training_job.name
training_job = sagemaker_session.sagemaker_client.describe_training_job(
    TrainingJobName=training_job_name
)

model_data_url = training_job["ModelArtifacts"]["S3ModelArtifacts"]
image = training_job["AlgorithmSpecification"]["TrainingImage"]

# model package request input object
create_model_package_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageDescription": "Test model package registered for integ test",
    "ModelApprovalStatus": ModelApprovalStatusEnum.PENDING_MANUAL_APPROVAL,
    "InferenceSpecification": {
        "Containers": [{"Image": image, "ModelDataUrl": model_data_url}],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"],
    },
}

model_pkg = sagemaker_session.sagemaker_client.create_model_package(
    **create_model_package_input_dict
)
print("Model package ARN:", model_pkg["ModelPackageArn"])

---
## Create Model Card<a name="ModelCard"></a>
Document your model package details in an Amazon SageMaker Model Card using the SageMaker Python SDK.

### 1. Collect model package details
Automatically collect basic model package information like model package ARN, model package group name, model package approval status, and model package's inference specification information.

In [ ]:
model_package_details = ModelPackage.from_model_package_arn(
    model_package_arn=model_pkg["ModelPackageArn"],
    sagemaker_session=sagemaker_session,
)

### 2. Initialize a model card
Initialize a model card with the model package details collected in the previous step. When associating model package to a model card, model card will try to auto discover information like training job details and evaluation job details only if there are information like model artifacts and model metrics available in model package. Additionally, it will also try to carry over some additional information like business details to this model card from the previously created the most recent model card that is associated with this particular model package group.

In [ ]:
model_card_name = "sample-model-card-with-model-package"
my_card = ModelCard(
    name=model_card_name,
    status=ModelCardStatusEnum.DRAFT,
    model_package_details=model_package_details,
    intended_uses=IntendedUses(
        purpose_of_model="Test model card.",
        intended_uses="Not used except this test.",
    ),
    sagemaker_session=sagemaker_session,
)

# Check auto-discovered data
print("Auto discovered training job details")
print("    arn: ", my_card.training_details.training_job_details.training_arn)
print(
    "    environment: ",
    my_card.training_details.training_job_details.training_environment.container_image,
)
print(
    "    metrics: ",
    [(m.name, m.value) for m in my_card.training_details.training_job_details.training_metrics],
)
print(
    "    hyper-parameters: ",
    [(h.name, h.value) for h in my_card.training_details.training_job_details.hyper_parameters],
)

my_card.create()
print(f"Model card {my_card.name} is successfully created with id {my_card.arn}")

time.sleep(
    3
)  # sleep 3s to wait for model card being populated in the search service which is required by information inheritance

#### Information inheritance
Additionally, new model card will also try to carry over some additional information like business details, intended uses, additional information to this model card from the previously created the most recent model card that is associated with this particular model package group. In this example, check out the intended uses that is automatically carried over from the previous model card.

In [ ]:
# create another model package under the same model package group as sample-model-card-with-model-package
create_model_package_input_dict2 = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageDescription": "Test model package registered for integ test",
    "ModelApprovalStatus": ModelApprovalStatusEnum.PENDING_MANUAL_APPROVAL,
    "InferenceSpecification": {
        "Containers": [{"Image": image, "ModelDataUrl": model_data_url}],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"],
    },
}

model_pkg2 = sagemaker_session.sagemaker_client.create_model_package(
    **create_model_package_input_dict2
)
model_package_details2 = ModelPackage.from_model_package_arn(
    model_package_arn=model_pkg2["ModelPackageArn"],
    sagemaker_session=sagemaker_session,
)

# create another model card with the new model package
model_card_name2 = "sample-model-card-with-model-package2"
my_card2 = ModelCard(
    name=model_card_name2,
    status=ModelCardStatusEnum.DRAFT,
    model_package_details=model_package_details2,
    sagemaker_session=sagemaker_session,
)

print(
    "Information carried over from the latest model card, i.e. sample-model-card-with-model-package, associated with the same model package group"
)
print("    Intended uses: ")
print("        purpose_of_model: ", my_card2.intended_uses.purpose_of_model)
print("        intended_uses: ", my_card2.intended_uses.intended_uses)

---
## Cleanup (Optional)<a name="Cleanup"></a>
Delete the following resources:
1. The model card
2. The model package
3. The model package group

In [ ]:
my_card.delete()

response = sagemaker_session.sagemaker_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name
)

for package in response["ModelPackageSummaryList"]:
    sagemaker_session.sagemaker_client.delete_model_package(
        ModelPackageName=package["ModelPackageArn"]
    )

sagemaker_session.sagemaker_client.delete_model_package_group(
    ModelPackageGroupName=model_package_group_name
)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/responsible_ai|sm-model_governance_model_card_with_model_package|sm-model_governance_model_card_with_model_package.ipynb)
